In [1]:
import pandas as pd
import numpy as np
import math
from importlib import reload
from regressionProject.models import feedforwardnet as ff
from regressionProject.models.feedforwardnet import NNregressor

2023-08-01 21:14:56.232617: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
forest_data = pd.read_csv('Downloads/forestfirestuff/forestfires.csv')

In [3]:
forest_data.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [4]:
colNames = ['month', 'day']

In [5]:
for col in colNames:
    if( forest_data[col].dtype == np.dtype('object')):
        dummies = pd.get_dummies(forest_data[col],prefix=col)
        forest_data = pd.concat([forest_data,dummies],axis=1)

        #drop the encoded column
        forest_data.drop([col],axis = 1 , inplace=True)

In [6]:
forest_data.fillna(forest_data.mean())

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,month_nov,month_oct,month_sep,day_fri,day_mon,day_sat,day_sun,day_thu,day_tue,day_wed
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,1,0,0,0,0,0,0
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,1,0,0,0,0,0,0,1,0
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,1,0,0,0,1,0,0,0,0
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,1,0,0,0,0,0,0
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,1,0,0,0
513,2,4,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,1,0,0,0
514,7,4,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,1,0,0,0
515,1,4,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,1,0,0,0,0


In [7]:
forest_data

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,month_nov,month_oct,month_sep,day_fri,day_mon,day_sat,day_sun,day_thu,day_tue,day_wed
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,1,0,0,0,0,0,0
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,1,0,0,0,0,0,0,1,0
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,1,0,0,0,1,0,0,0,0
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,1,0,0,0,0,0,0
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,1,0,0,0
513,2,4,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,1,0,0,0
514,7,4,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,1,0,0,0
515,1,4,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,1,0,0,0,0


In [8]:
from sklearn.model_selection import train_test_split

In [56]:
train, test = train_test_split(forest_data, train_size = 0.8)

In [57]:
y_train = train[['X', 'Y']]
X_train = train.drop(['X', 'Y'], axis = 1)
X_train.shape, y_train.shape

((413, 28), (413, 2))

In [122]:
reload(ff)

<module 'regressionProject.models.feedforwardnet' from '/Users/samdulin/regressionProject/models/feedforwardnet.py'>

In [123]:
nn = ff.NNregressor()

In [124]:
nn.fit(X_train = X_train, 
              y_train = y_train)

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_120 (Dense)           (None, 128)               5248      
                                                                 
 dense_121 (Dense)           (None, 256)               33024     
                                                                 
 dense_122 (Dense)           (None, 256)               65792     
                                                                 
 dense_123 (Dense)           (None, 256)               65792     
                                                                 
 dense_124 (Dense)           (None, 2)                 514       
                                                                 
Total params: 170,370
Trainable params: 170,370
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1400


/Users/samdulin/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


12/12 [==============================] - 0s 8ms/step - loss: 7.8346 - mean_absolute_error: 2.2586 - val_loss: 9.4238 - val_mean_absolute_error: 2.1916
Epoch 2/1400
12/12 [==============================] - 0s 3ms/step - loss: 4.6164 - mean_absolute_error: 1.6492 - val_loss: 5.2043 - val_mean_absolute_error: 1.7114
Epoch 3/1400
12/12 [==============================] - 0s 3ms/step - loss: 4.0022 - mean_absolute_error: 1.6102 - val_loss: 5.6220 - val_mean_absolute_error: 1.8768
Epoch 4/1400
12/12 [==============================] - 0s 3ms/step - loss: 3.8986 - mean_absolute_error: 1.6101 - val_loss: 5.2497 - val_mean_absolute_error: 1.6975
Epoch 5/1400
12/12 [==============================] - 0s 3ms/step - loss: 3.3319 - mean_absolute_error: 1.4453 - val_loss: 4.3956 - val_mean_absolute_error: 1.6771
Epoch 6/1400
12/12 [==============================] - 0s 3ms/step - loss: 3.4062 - mean_absolute_error: 1.4694 - val_loss: 4.3580 - val_mean_absolute_error: 1.6197
Epoch 7/1400
12/12 [=========

{'loss': [7.834582328796387,
  4.616390705108643,
  4.002244472503662,
  3.8985536098480225,
  3.3319385051727295,
  3.4062387943267822,
  3.226663589477539,
  3.5385446548461914,
  3.6088900566101074,
  3.424250364303589,
  3.377794027328491,
  3.205325126647949,
  3.2404236793518066,
  3.256591558456421,
  3.1370465755462646,
  3.2033050060272217,
  3.1928365230560303,
  3.2714295387268066,
  3.060011148452759,
  3.040142059326172,
  3.1323983669281006,
  3.045898675918579,
  3.2284786701202393,
  3.1288506984710693,
  3.162006139755249,
  3.1727945804595947,
  3.010324716567993,
  3.1360700130462646,
  3.0320262908935547,
  2.973876714706421,
  3.0301711559295654,
  2.969475746154785,
  2.978494644165039,
  3.0617258548736572,
  3.083752155303955,
  2.9506053924560547,
  2.908310651779175,
  3.094453811645508,
  3.028844118118286,
  3.0205330848693848,
  3.3475451469421387,
  2.9828224182128906,
  2.938652276992798,
  2.9448344707489014,
  3.086925506591797,
  3.071242570877075,
  3

In [58]:
y_test = test[['X', 'Y']]
X_test = test.drop(['X', 'Y'], axis = 1)
X_test.shape, y_test.shape

((104, 28), (104, 2))

In [126]:
y_pred = nn.predict(X_test)

4/4 [==============================] - 0s 1ms/step


In [13]:
y_test = np.array(y_test)

In [46]:
sk.metrics.mean_squared_error(y_test, y_pred)

3.7237388623130325

In [47]:
import sklearn as sk
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [40]:
regressor = RandomForestRegressor()

In [41]:
regressor.fit(X_train, y_train)

RandomForestRegressor()

In [43]:
y_pred = regressor.predict(X_test)

In [ ]:
sk.metrics.mean_squared_error(y_test, y_pred)

In [59]:
y_train1 = y_train.iloc[:, 0]
y_train2 = y_train.iloc[:, 1]
y_test1 = y_test.iloc[:, 0]
y_test2 = y_test.iloc[:, 1]

In [60]:
regressor1 = SVR(kernel = 'rbf')
regressor2 = SVR(kernel = 'rbf')

In [61]:
regressor1.fit(X_train, y_train1)
regressor2.fit(X_train, y_train2)

SVR()

In [62]:
X_test

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,month_apr,...,month_nov,month_oct,month_sep,day_fri,day_mon,day_sat,day_sun,day_thu,day_tue,day_wed
478,93.7,101.3,423.4,14.7,18.2,82,4.5,0.0,2.21,0,...,0,0,0,0,0,0,1,0,0,0
421,95.2,217.7,690.0,18.0,30.8,19,4.5,0.0,0.00,0,...,0,0,0,0,0,0,0,0,0,1
239,81.9,3.0,7.9,3.5,13.4,75,1.8,0.0,0.00,1,...,0,0,0,0,0,0,1,0,0,0
243,91.8,175.1,700.7,13.8,21.9,73,7.6,1.0,0.00,0,...,0,0,0,0,0,0,1,0,0,0
385,91.6,181.3,613.0,7.6,20.9,50,2.2,0.0,16.00,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,93.0,103.8,316.7,10.8,26.4,35,2.7,0.0,10.08,0,...,0,0,0,0,0,0,0,1,0,0
343,91.5,130.1,807.1,7.5,15.9,51,4.5,0.0,2.18,0,...,0,0,1,0,1,0,0,0,0,0
383,91.6,248.4,753.8,6.3,20.5,58,2.7,0.0,42.87,0,...,0,0,0,0,0,0,0,1,0,0
37,90.0,41.5,682.6,8.7,11.3,60,5.4,0.0,0.00,0,...,0,1,0,1,0,0,0,0,0,0


In [63]:
y_pred1 = regressor1.predict(X_test)
y_pred2 = regressor2.predict(X_test)

In [78]:
y_pred1 = np.expand_dims(y_pred1, axis = 1)
y_pred2 = np.expand_dims(y_pred2, axis = 1)

In [79]:
y_pred = np.concatenate((y_pred1, y_pred2), axis = 1)

In [82]:
sk.metrics.mean_squared_error(y_test, y_pred)

3.4987804431181826